# THE GOAL

The goal is to take Watson's role and using the intel (the data in the supplied files) from the police, Interpol, and undercover agents about Europe's criminals to identify the name behind which Moriarty is hiding. 


# SOLUTION

# PART 1
-Watson, just like our grand-grand-fathers we are again after Moriarty. 

We need to catch him. H-mmm... I need to be careful here - maybe it is not him, maybe it is her. All we know is 
that someone is masterminding unlawful activities and planning something bad. The Interpol agents, with the help of my boys, collected information that should provide us the clues to determine the name Moriarty's is hiding brhind and arrest him.

-I have a number of .csv and .txt files about criminal activity and high-profile suspicious sales that were sent over from our neighbors: France, Germany, Netherlands, and our own MI-6 in the United Kingdom.

So, the first task would be to combine the data into one table. I requested info on the name, alias, and the location of the last known whereabouts, as latitude and longitude, but since the data comes from all around the Europe they might have named the columns differently.

I am thinking that adding the country to the data might be helpful in our future analysis.

Lastly, from my correspondence with our undercover agents, all the activity seems to be happening around major financial centers. If the city names are not in the data, I suppose you can extract it based on the latitude and logitude. Mmmm... And a map of course, unless your knowledge of Europe's geography is excepitonal. 





Text:
Tasks:
1. Read in data from the files into a separate dataframe and add the country name ('country' column).
2. Identify the city around which the criminals operate. Add it to the dataframe ('city' column).
3. Concatenate dfs into a single dataframe with the four original columns renamed to: [name, alias, latitude, longitude]
4. Fill NAs in aliases with an empty string.


In [1]:
import pyspark
print(pyspark.__version__)

3.0.1


In [2]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F

# from https://datascience.stackexchange.com/questions/11356/merging-multiple-data-frames-row-wise-in-pyspark
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Python Spark SQL basic example") \
    .config("spark.sql.warehouse.dir", "hdfs://namenode/sql/metadata/hive")\
    .enableHiveSupport()\
    .getOrCreate()
# .config("spark.sql.warehouse.dir", "hdfs://namenode/sql/metadata/hive") \

In [3]:
# select database to use and create tables in
spark.sql("show databases").show(10, False)
spark.sql("use default")  # 'default' is a pre-created database where we can create tables
# (we could have skipped this statement but it makes it more explicity which database we use)

+---------+
|namespace|
+---------+
|default  |
+---------+



DataFrame[]

In [4]:


# we could also create our tables in our own custom database using:
# spark.sql("create database moriarty_db")
# spark.sql("use moriarty_db")
spark.sql("show tables").show(10, False)

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [5]:
%ls data

ls: data: No such file or directory


In [7]:
# get the data

def rename_cols(df, new_col_names):
    """"""
    for col, new_col in zip(df.columns, new_col_names):
        df = df.withColumnRenamed(col, new_col)
        
    return df

#explore the dataframes: column names, shapes and combine into a single dataframe
country_list = ["United Kingdom", "Germany", "Netherlands", "France"]
dfs_dict = {}
for country_ in country_list:
    file_name = "./data/criminals_{}.csv".format(country_)
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    print("Country: {}, rows: {}".format(country_, df.count()))
    new_col_names = ["id", "name", "alias", "latitude", "longitude"]
    df = rename_cols(df, new_col_names)
    df = df.withColumn('country', F.lit(country_))
    country_ = "_".join(country_.split())  # 'United Kingdom' as space in it and thus is an illigal table name
    df.registerTempTable("criminals_{}".format(country_))


AnalysisException: Path does not exist: file:/Users/vk/Documents/Python/holmes_moriarty_sql/src/from_downloads/data/criminals_United Kingdom.csv;

In [ ]:
def unionAll(dfs):
    return reduce(DataFrame.unionAll, dfs)

df_criminals_combined = unionAll(list(dfs_dict.values()))
print("Rows in combined df: {}".format(df_criminals_combined.count()))

In [ ]:
# save as ORC file (a popular data format in big data management)
df_criminals_combined.cache()
df_criminals_combined.coalesce(1).write.orc("./sql_data/criminals", mode='overwrite') # doesn't have 'orc' extension as it is a folder
# the file with extension '.orc' will be inside it

In [ ]:
# check schema before casting and saving
df_criminals_combined.printSchema()

In [ ]:
# often to insure that datatypes are compatible for retrieval via Hive we need to explicitly define data types

# import data types to cast data and define schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, \
                                DecimalType, FloatType

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("alias", StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True),
    StructField("country", StringType(), True)
])

# apply schema
df_criminals_combined_new_schema = spark.createDataFrame(df_criminals_combined.rdd, schema=schema)
df_criminals_combined_new_schema.printSchema()

In [ ]:
# create a table that defines the data (including the data location)

# drop table if we created it before and define the table by registering 'table definition'
spark.sql("drop table if exists criminals")

# the hive datatypes should be appropriate (not necessarily identically named to spark datatypes)

# 'EXTERNAL' (also could be 'external') makes sure that if the table is dropped (deleted) the data remains
spark.sql("""CREATE EXTERNAL TABLE criminals (
 id int,
 name string,
 alias string,
 latitude float,
 longitude float,
 country string
    )
STORED AS ORC
LOCATION '/Users/vk/Documents/Python/holmes_moriarty_sql/src/sql_data/criminals'
"""
)

In [ ]:
spark.sql("show tables").show()

In [ ]:
criminals_df = spark.sql("select * from criminals")
# print("Table (read-in) count: {}".format(criminals_df.count()))
criminals_df.show(10, False)

In [ ]:
criminals_df.printSchema()

In [ ]:
# calculate mean latitude and longitude to identify the major financial centers (cities)
# (copy and paste the lat, lon values into Google Maps)
# dataframe.filter(df['salary'] > 100000).agg({"age": "avg"})

spark.sql("""select country, 
                    AVG(latitude) as avg_lat, 
                    AVG(longitude) as avg_lon
                    from criminals
                    group by country
                    order by country""").show(10, False)


In [ ]:
spark.sql("""select country,
                    ROUND(AVG(latitude), 4) as avg_lat,
                    ROUND(AVG(longitude), 4) as avg_lon
                from criminals
                 group by country
                 order by country""").show(10, False)

In [ ]:
# add the city name to the df

#it can be done using a series of if/else statements, such as 'if country_ == 'France': city = 'Paris', etc. OR
# using a dictionary as below:
country_city_dict = {"United Kingdom": "London", "Germany": "Frankfurt", "Netherlands": "Amsterdam", "France": "Paris"}
country_city_dict


In [ ]:
spark.sql("""select *, 
                case 
                    when country = 'United Kingdom' then 'London'
                    when country = 'France' then 'Paris'
                    when country = 'Germany' then 'Frankfurt'
                    when country = 'Netherlands' then 'Amsterdam'
                end as city
            from criminals""").show(10, False)

In [ ]:
# fill Nas with empty string 
# we'll also assign this new data to a variable name for saving and creating a new table to use later
# (note that 'show' method is moved to the spark dataframe)
criminals_with_city = spark.sql("""select id, name,
                case
                    when alias is null then ''
                    else alias
                end as alias,
                country,
                case 
                    when country = 'United Kingdom' then 'London'
                    when country = 'France' then 'Paris'
                    when country = 'Germany' then 'Frankfurt'
                    when country = 'Netherlands' then 'Amsterdam'
                end as city
            from criminals""")

criminals_with_city.show(10, False)

In [ ]:
criminals_with_city.printSchema()

In [ ]:
criminals_with_city.cache()
criminals_with_city.coalesce(1).write.orc("./sql_data/criminals_with_city", mode='overwrite') # doesn't have 'orc' extension as it is a folder
# the file with extension '.orc' will be inside it

In [ ]:
spark.sql("drop table if exists criminals_with_city")

spark.sql("""CREATE EXTERNAL TABLE criminals_with_city (
 id int,
 name string,
 alias string,
 country string,
 city string)
STORED AS ORC
LOCATION '/Users/vk/Documents/Python/holmes_moriarty_sql/src/sql_data/criminals_with_city'
"""
)


In [ ]:
# check that the data is readable
spark.sql("select * from criminals_with_city").show(10, False)

# Task 2
Add crime_type and profit info to criminals. 
#(merge/join) criminals table with the crime type and profit information.

- Great, Watson! 
- Now we need to know what everyone of those supspects did wrong, that is the crime type, and desirably, how much they profited from it: Moriarty is not a small fish. He is in the category with th largest total sales.

- You'll need to add the crime type and the profit from the files to the table you already put together. Be mindful of the file types. I also believe that the separator in these file maybe different from the files you used previously.
-Moriarty made one of the top 5 sales last year. He is not stupid for nicknames, I am pretty sure he doesn't have an alias.


# Solution (task 2)

In [ ]:
# union(concatenate) files for the latest crime dates

country_list = ["United Kingdom", "Germany", "Netherlands", "France"]

for country_ in country_list:
    file_name = "./data/crime_type_profit_{}.txt".format(country_)
    df = spark.read.csv(file_name, header=True, sep=" ")
    print("rows: {}".format(df.count()))
    df = df.withColumn('country', F.lit(country_))
    country_ = "_".join(country_.split())  # 'United Kingdom' as space in it and thus is an illigal table name
    df.registerTempTable("crime_profit_{}".format(country_))




In [ ]:
spark.sql("use default")
spark.sql("show tables").show(10, False)

In [ ]:
spark.sql("select * from criminals_france").show(10, False)

In [ ]:
spark.sql("select * from criminals_with_city").show(10, False)

In [ ]:
#combine all dataframes into one
df_crime_type_profit = unionAll(list(dfs_dict.values()))
print(list(df_crime_type_profit.columns))
df_crime_type_profit.show(4, False)


In [ ]:
df_crime_type_profit.printSchema()

In [ ]:
# profit is a string - which is incorrect -> the schema needs to be redifined
# or the column recasted

In [ ]:
df_crime_type_profit = df_crime_type_profit.withColumn('profit', F.col('profit').cast(IntegerType()))

In [ ]:
df_crime_type_profit.printSchema()

In [ ]:
df_crime_type_profit.cache()
df_crime_type_profit.coalesce(1).write.orc("./sql_data/crime_profit", mode="overwrite")

In [ ]:
spark.sql("drop table if exists crime_profit")

spark.sql("""CREATE EXTERNAL TABLE crime_profit (
  name string,
  crime_type string,
  profit int,
  country string)
STORED AS ORC
LOCATION '/Users/vk/Documents/Python/holmes_moriarty_sql/src/sql_data/crime_profit'
"""
)

In [ ]:
spark.sql("select * from crime_profit").show(10, False)

In [ ]:
spark.sql("""select  a.id, a.name, a.alias, b.crime_type, b.profit, b.country
            from criminals a
            left join crime_profit b
                on a.name = b.name and a.country = b.country""").show(10, False)

In [ ]:
spark.sql("""select count(*) as row_count from (
                select  a.id, a.name, a.alias, b.crime_type, b.profit, b.country
                from criminals a
                left join crime_profit b
                    on a.name = b.name and a.country = b.country)""").show(10, False)

In [ ]:
# order by profit (descending) and 
# cast profit as int (to keep the same with pyspark notebook; not necessary here)
spark.sql("""select  a.id, a.name, a.alias, b.crime_type, CAST(b.profit AS INT), b.country
            from criminals a
            left join crime_profit b
                on a.name = b.name and a.country = b.country
            order by profit DESC""").show(10, False)

In [ ]:
# find the most profitable crime type

spark.sql("""select crime_type, sum(profit) as total_profit
            from criminals a
            left join crime_profit b
                on a.name = b.name and a.country = b.country
            group by crime_type
            order by total_profit DESC""").show(10, False)

In [ ]:

spark.sql("show tables").show(10)

In [ ]:
# find the country where with the highest weapons sales

spark.sql("""select a.country, sum(profit) as total_profit
            from criminals a
            left join crime_profit b
                on a.name = b.name and a.country = b.country
            where crime_type = 'weapons sale'
            group by a.country
            order by total_profit DESC
            """).show(10, False)

In [ ]:
df_city_profit.show(3)

In [ ]:
# Show top 5 salesmen in the selected country
df_large_sales_alias_null = df_city_profit.where("country = '{}' and alias = '' and crime_type = '{}'".format(top_country, crime_type_big_sales))\
                                            .orderBy("profit", ascending = False)

df_large_sales_alias_null.show(5)

# PART 3

Add date (last deal date) Moriarty does not deal on Sundays

In [ ]:
id_dates = spark.read.csv("./data/id_dates.csv", header=True, inferSchema=True)
print("id_dates shape: {}".format(id_dates.count()))
id_dates.show(4)

In [ ]:
df_selected_with_dates = df_city_profit.join(id_dates, on=["id", "country"], how="left")
print(df_selected_with_dates.count())
df_selected_with_dates.show(3)

In [ ]:
df_selected_with_dates.printSchema()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType, StringType


def weekday(date):
    """ Generate day of the week based on date (as string or as datetime object)"""
    
    if isinstance(date, str):
        from datetime import datetime
        
        date = datetime.strptime(date, "%Y-%m-%d")  # change the format if necessary
        
    return date.strftime("%A")


weekday_udf = udf(weekday, StringType())

# conversion to DateType is not necessary as it is handled inside the function
# here it is offered as an example of re-casting
df_selected_with_dates = df_selected_with_dates.withColumn("date", F.col("date").cast(DateType()))

df_selected_with_dates = df_selected_with_dates.withColumn("weekdate", weekday_udf("date").alias("weekday"))
df_selected_with_dates.show(10)

In [ ]:
crime_type_big_sales

In [ ]:
# Show top 5 salesmen in the selected country
df_final = df_selected_with_dates.where("""country = '{}' 
                                            and alias = '' 
                                            and crime_type = '{}'
                                            and weekday != 'Sunday'
                                       """.format(top_country, crime_type_big_sales))

df_final.show(5)

In [ ]:
moriarty_name =  df_final.select("name").collect()[0][0]
print("The name Moriarty is hiding behind: {}".format(moriarty_name))